# Project Five 
# Game of Thrones - Season Six

## Importing and Cleaning

In [1]:
import numpy as np
import pandas as pd
import nltk
from nltk.tag import pos_tag
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from textblob import TextBlob
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import NMF, LatentDirichletAllocation
import gensim
from nltk.tag import pos_tag
import re

Using Theano backend.


In [2]:
pd.set_option('display.max_columns', 135)
pd.set_option('display.max_rows', 135)

In [3]:
df_s6 = pd.read_json('/Users/zoemarkovits/Metis/project_five_got/scrape/season_six.json')

In [4]:
df_s6[['title','script_text']]

,title,script_text
0,[The Red Woman (Script)],"[\n, \n, CUT TO: CASTLE BLACK - DAVOS’ CHAMBER..."
1,[Home (Script)],"[INT. GREAT WEIRWOOD TREE - CAVE, \n, \n, \n, ..."
2,[Oathbreaker (Script)],"[\n, \nDAVOS: Easy, easy, Easy, \n, \n, \n, \..."
3,[Book of the Stranger (Script)],"[INT. CASTLE BLACK - JON SNOW’S ROOM, \n, \n, ..."
4,[The Door (Script)],"[INT. CASTLE BLACK - SANSA’S ROOM, \n, \n, \n,..."
5,[Blood of My Blood (Script)],"[\n, \nBRAN: Meera. They found us., \n, \n, \n..."
6,[The Broken Man (Script)],"[\n, \nMAN: That’s it. Get it to the top., \n,..."
7,[No One (Script)],"[ The Bloody Hand. , \n, \nLADY CRANE: My son...."
8,[Battle of the Bastards (Script)],"[\n, \nTYRION: Despite appearances, I think yo..."
9,[The Winds of Winter (Script)],"[\n, \n, CUT TO: RED KEEP - TOMMEN’S CHAMBER, ..."


In [5]:
df_s6['title'] = df_s6['title'].map(lambda x: str(x)[3:-11])

In [6]:
df_s6.title

0             The Red Woman
1                      Home
2               Oathbreaker
3      Book of the Stranger
4                  The Door
5         Blood of My Blood
6            The Broken Man
7                    No One
8    Battle of the Bastards
9       The Winds of Winter
Name: title, dtype: object

In [7]:
names6_01 = ['NIGHT’S WATCHMAN','DAVOS','DOLOROUS EDD','MELISANDRE','MALE VOICE #1','MALE VOICE #2','ALLISER THORNE',
             'NIGHT’S WATCHMEN #1','NIGHT’S WATCHMEN #2','RAMSAY','MAESTER WOLKAN','ROOSE','THEON','SANSA',
             'BOLTON OFFICER','BRIENNE','PODRICK','HANDMAIDEN','CERSEI','JAIME','SEPTA UNELLA','MARGAERY',
             'HIGH SPARROW','DORAN','ELLARIA','MAESTER','TRYSTANE','OBARA','NYMERIA','TYRION','VARYS','RED PRIEST',
             'DAARIO','JORAH','BLOODRIDER #1','BLOODRIDER #2','WIFE #1','WIFE #2','KHAL MORO','BLOODRIDER #3',
             'BLOODRIDER #4','DAENERYS','ARYA','WAIF','NIGHT’S WATCHMAN #1','NIGHT’S WATCHMAN #2']

names6_02 = ['YOUNG RODRICK','BRAN','YOUNG NED','THREE-EYED RAVEN','YOUNG BENJEN','YOUNG LYANNA','YOUNG HODOR','NAN',
             'HODOR','MEERA','CHILD OF THE FOREST','ALLISER THORNE','DAVOS','DOLOROUS EDD','TORMUND','STORYTELLER',
             'CERSEI','KING’S SOLDIER','TOMMEN','JAIME','HIGH SPARROW','TYRION','VARYS','GREY WORM','MISSANDEI',
             'WAIF','ARYA','JAQEN','LORD KARSTARK','ROOSE','RAMSAY','MAESTER WOLKAN','LADY WALDA','RAMSEY','BRIENNE',
             'SANSA','THEON','YARA','BALON','HOODED FIGURE','EURON','AERON','ALL','MELISANDRE']

names6_03 = ['DAVOS','JON SNOW','MELISANDRE','TORMUND','DOLOROUS EDD','GILLY','SAMWELL','BRAN','THREE-EYED RAVEN',
             'ARTHUR','NED','BLOODRIDER','DAENERYS','KHAL MORO','DOTHRAKI WOMAN','DOTHRAKI MATRON','VARYS','WOMAN',
             'VALA','TYRION','GREY WORM','MISSANDEI','QYBURN','ARTHUR','LITTLE BIRD','CERSEI','JAIME',
             'MAESTER PYCELLE','KEVAN','MACE TYRELL','LADY OLENNA','HIGH SPARROW','TOMMEN','WAIF','ARYA','JAQEN',
             'RAMSAY','LORD KARSTARK','SMALLJON','BOWEN MARSH','OTHELL YARWICK','ALLISER THRONE']

names6_04 = ['DOLOROUS EDD','JON SNOW','MAN','MAN #2','SANSA','DAVOS','MELISANDRE','BRIENNE','LITTLEFINGER','ROBIN',
             'YOHN ROYCE','TYRION','GREY WORM','MISSANDEI','YEZZAN','RAZDAL','BELICHO','MAN #1','MAN #2','DAARIO',
             'JORAH','DOTHRAKI #1','LEADER','LHAZAREEN WOMAN','DAENERYS','HIGH SPARROW','MARGAERY','LORAS',
             'MAESTER PYCELLE','CERSEI','TOMMEN','LADY OLENNA','JAIME','KEVAN','YARA','THEON','RAMSAY','OSHA','MAN',
             'TORMUND','KHAL MORO','KHAL #1','KHAL #2','KHAL #3','KHAL #4']

names6_05 = ['SANSA','MAN','LITTLEFINGER','BRIENNE','WAIF','ARYA','JAQEN','IZEMBARO','CLARENZO','LADYC RANE',
             'CAMELLO','BOBONO','LADY CRANE','BIANCA','BRAN','LEAF','AERON','YARA','MAN #1','ALL','MAN #2','THEON',
             'EURON','AERSON','DAENERYS','JORAH','VARYS','GREY WORM','TYRION','MISSANDEI','KINVARA',
             'THREE-EYED RAVEN','MEERA','HODOR','DAVOS','JON SNOW','DOLOROUS EDD','RICKARD','YOUNG NED','NAN',
             'YOUNG HODOR']

names6_06 = ['BRAN','MEERA','RIDER','GILLY','SAM','MELESSA','TALLA','TOMMEN','HIGH SPARROW','MARGAERY','DICKON',
             'RANDYLL','CLARENZO','ALL','BOBONO','LADY CRANE','IZEMBARO','CAMELLO','ARYA','BIANCA','WAIF','JAQEN',
             'MACE','JAIME','CAPTAIN','WOMAN','OLENNA','WALDER','BLACK WALDER','LOTHAR','CERSEI','BENJEN','DAARIO',
             'DAENERYS']

names6_07 = ['MAN','MAN #2','RAY','SANDOR','HIGH SPARROW','MARGAERY','OLENNA','DIM DALBA','TORMUND','JON SNOW',
             'WUN WUN','CERSEI','BRONN','JAIME','LOTHAR','BLACK WALDER','BRYNDEN','LYANNA','SANSA','DAVOS','ROBETT',
             'THEON','YARA','LEM','ARYA','WOMAN']

names6_08 = ['LADY CRANE','ARYA','MAN','GATINS','YOUNG MAN','YOUNG MAN #2','SANDOR','RED PRIESTESS','TYRION','VARYS',
             'QYBURN','CERSEI','LANCEL','PODRICK','BRIENNE','BRONN','JAIME','BRYNDEN','KEVAN','TOMMEN','CROWD',
             'MISSANDEI','GREY WORM','EDMURE','GUARD CAPTAIN','GUARD','BLACK WALDER','BERIC','THOROS','SANDOR',
             'LEM','WAIF','JAQEN']

names6_09 = ['TYRION','DAENERYS','RAZDAL','YEZZAN','GREY WORM','MISSANDEI','BELICHO','JON SNOW','SANSA','RAMSAY',
             'DAVOS','TORMUND','MELISANDRE','THEON','YARA','KARSTARK','SMALLJON','ALL','MAN','MAN #2']

names6_10 = ['ATTENDANT','MISTRESS','PYCELLE','GIRL','HIGH SPARROW','LORAS','MACE','MARGAERY','TOMMEN','LANCEL',
             'QYBURN','CERSEI','SEPTA UNELLA','WALDER','MEN','BRONN','JAIME','SAM','MAESTER','JON SNOW','MELISANDRE',
             'DAVOS','SANSA','OLENNA','ELLARIA','OBARA','NYMERIA','VARYS','DAARIO','DAENERYS','TYRION','HANDMAIDEN',
             'ARYA','PETYR','BENJEN','MEERA','BRAN','LYANNA','NED','VALE KNIGHT','TORMUND','MAN','MANDERLY','GLOVER',
             'ALL']

### Episode One

In [8]:
#df_s6.script_text[0]

In [9]:
script_text6_01 = df_s6.script_text[0]
print type(script_text6_01)

#script_text6_01

<type 'list'>


In [10]:
script_split = [i.split('\n') for i in script_text6_01]
#script_split

In [11]:
string_script_text = ' '

for i in script_split:
    for e in i:
        string_script_text = string_script_text + ' ' + e

In [12]:
#string_script_text

In [13]:
words = word_tokenize(string_script_text)
#words

In [14]:
sentences = sent_tokenize(string_script_text)
#sentences

In [15]:
dialogue = []

for sentence in sentences:
    if sentence.split(':')[0] in (names6_01):
        dialogue.append(sentence)

/Users/zoemarkovits/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:4: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal
  after removing the cwd from sys.path.


In [16]:
#dialogue

In [17]:
import pickle
with open('/Users/zoemarkovits/Metis/project_five_got/Dialogue_PKLs/s601.pkl', 'wb') as f:
    pickle.dump(dialogue, f)

In [18]:
from collections import defaultdict

speakers = defaultdict(list)
for sentence in dialogue:
    speakers[sentence.split(':')[0]].append(sentence.split(':')[1])

#speakers

In [19]:
speakers['DAVOS']

[u' Help me get him inside         CUT TO',
 u' We need all the help we can get        MELISANDRE (O.S.',
 u' You\u2019re not the only ones who owe your lives to Jon Snow        DOLOROUS EDD',
 u' I\u2019m not much of a hunter  I\u2019ll need some food if I\u2019m gonna make it south without starving    ALLISER THORNE',
 u' You haven\u2019t seen her do what I\u2019ve seen her do     CUT TO']

In [20]:
episode_one = pd.DataFrame.from_dict(speakers, orient="index")
print episode_one.shape

(24, 6)


In [21]:
#from numpy import nan
#episode_one.fillna(value=str(0), inplace=True)

In [30]:
#episode_one.info()

In [31]:
#episode_one.head()

In [24]:
def remove_stops(sentence):
    characters = ['?','!','.',',',':',';']
    stoplist = stopwords.words('english') + characters
    try: 
        new_sentence = re.sub(r'[^a-zA-Z ]','', sentence)
        new_sentence = new_sentence.lower()
        new_sentence = new_sentence.split()
        new_sentence = [word for word in new_sentence if word not in stoplist]
        new_sentence = ' '.join(new_sentence)
        return new_sentence
    except:
        return sentence   

In [25]:
test = episode_one[0].apply(remove_stops)
#test

In [26]:
for column in episode_one.columns:
    for x in range(len(episode_one[column])):
        episode_one[column][x] = remove_stops(episode_one[column][x])

In [27]:
#episode_one.info()

In [28]:
episode_one.index = episode_one.index.str.title()

In [29]:
episode_one.head()

,0,1,2,3,4,5
Davos,help get inside cut,need help get melisandre os,youre ones owe lives jon snow dolorous edd,im much hunter ill need food im gonna make sou...,havent seen ive seen cut,None
Male Voice #2,never,None,None,None,None,None
Jaime,mother cersei,None,None,None,None,None
Tyrion,never going fix whats wrong city top foot pyra...,hard know queen popular meereen used varys,possibly daenerys targaryen took away valuable...,problem tyrion,None,None
Wife #1,told witch cut head khal moro,None,None,None,None,None


### Episode Two

In [32]:
#df_s6.script_text[1]

In [33]:
script_text6_02 = df_s6.script_text[1]
print type(script_text6_02)

#script_text6_02

<type 'list'>


In [34]:
script_split2 = [i.split('\n') for i in script_text6_02]
#script_split2

In [35]:
string_script_text2 = ' '

for i in script_split2:
    for e in i:
        string_script_text2 = string_script_text2 + ' ' + e

In [36]:
#string_script_text2

In [37]:
words2 = word_tokenize(string_script_text2)
#words2

In [38]:
sentences2 = sent_tokenize(string_script_text2)
#sentences2

In [39]:
dialogue2 = []

for sentence in sentences2:
    words=sentence.split(':')[0]
    if len(words.split())<=2:
        if words in (names6_02):
            dialogue2.append(sentence)

/Users/zoemarkovits/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:6: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal
  


In [40]:
#dialogue2

In [41]:
import pickle
with open('/Users/zoemarkovits/Metis/project_five_got/Dialogue_PKLs/s602.pkl', 'wb') as f:
    pickle.dump(dialogue2, f)

In [42]:
from collections import defaultdict

speakers2 = defaultdict(list)
for sentence in dialogue2:
    speakers2[sentence.split(':')[0]].append(sentence.split(':')[1])

#speakers2

In [43]:
speakers2['TYRION']

[u' Dragons do not do well in captivity    MISSANDEI',
 u' That\u2019s what I do  I drink and I know things        TYRION',
 u' They will  It\u2019s time they had their supper     CUT TO']

In [44]:
episode_two = pd.DataFrame.from_dict(speakers2, orient="index")
print episode_two.shape

(19, 4)


In [45]:
#from numpy import nan
#episode_two.fillna(value=str(0), inplace=True)

In [46]:
#episode_two.info()

In [47]:
#episode_two.head()

In [48]:
test2 = episode_two[0].apply(remove_stops)
#test2

In [49]:
for column in episode_two.columns:
    for x in range(len(episode_two[column])):
        episode_two[column][x] = remove_stops(episode_two[column][x])

In [50]:
#episode_two.info()

In [51]:
episode_two.index = episode_two.index.str.title()

In [52]:
episode_two.head()

,0,1,2,3
Young Benjen,dont know young lyanna,hes got giants blood young ned,None,None
Theon,would taken way wall would died get theon,home ext,None,None
Jaime,fail sometimes tommen,go see mother ask forgive high sparrow,broke sacred oath stabbed king back killed cou...,oh gods wont mind theyve spilled blood rest us...
Roose,jon snow bastard stark ramsay,None,None,None
Tommen,course jaime,faith militant seized margaery,gold cersei,None


### Episode Three

In [53]:
#df_s6.script_text[2]

In [54]:
script_text6_03 = df_s6.script_text[2]
print type(script_text6_03)

#script_text6_03

<type 'list'>


In [55]:
script_split3 = [i.split('\n') for i in script_text6_03]
#script_split3

In [56]:
string_script_text3 = ' '

for i in script_split3:
    for e in i:
        string_script_text3 = string_script_text3 + ' ' + e

In [57]:
#string_script_text3

In [58]:
words3 = word_tokenize(string_script_text3)
#words3

In [59]:
sentences3 = sent_tokenize(string_script_text3)
#sentences3

In [60]:
dialogue3 = []

for sentence in sentences3:
    words=sentence.split(':')[0]
    if len(words.split())<=2:
        if words in (names6_03):
            dialogue3.append(sentence)

In [61]:
#dialogue3

In [62]:
import pickle
with open('/Users/zoemarkovits/Metis/project_five_got/Dialogue_PKLs/s603.pkl', 'wb') as f:
    pickle.dump(dialogue3, f)

In [63]:
from collections import defaultdict

speakers3 = defaultdict(list)
for sentence in dialogue3:
    speakers3[sentence.split(':')[0]].append(sentence.split(':')[1])

#speakers3

In [64]:
speakers3['JON SNOW']

[u' They stabbed me  Olly...he put a knife in my heart  I shouldn\u2019t be here    DAVOS',
 u' I think so  Hold off on burning my body for now        DOLOROUS EDD',
 u' Wear it  Burn it  Whatever you want  You have Castle Black                 CREDITS']

In [65]:
episode_three = pd.DataFrame.from_dict(speakers3, orient="index")
print episode_three.shape

(22, 4)


In [66]:
#from numpy import nan
#episode_three.fillna(value=str(0), inplace=True)

In [67]:
#episode_three.info()

In [68]:
#episode_three.head()

In [69]:
test3 = episode_three[0].apply(remove_stops)
#test3

In [70]:
for column in episode_three.columns:
    for x in range(len(episode_three[column])):
        episode_three[column][x] = remove_stops(episode_three[column][x])

In [71]:
episode_three.info()

<class 'pandas.core.frame.DataFrame'>
Index: 22 entries, DAVOS to MAESTER PYCELLE
Data columns (total 4 columns):
0    22 non-null object
1    13 non-null object
2    9 non-null object
3    4 non-null object
dtypes: object(4)
memory usage: 880.0+ bytes


In [72]:
episode_three.index = episode_three.index.str.title()

In [73]:
episode_three.head()

,0,1,2,3
Davos,dead youre thats completely fucking mad seems ...,dont know maybe well never know matter,None,None
Melisandre,lord let come back reason stannis prince promi...,None,None,None
Smalljon,bastard jon snow led army wildlings past wall ...,need help colder gets farther south goat fucke...,father honored tradition knelt robb stark call...,rickon stark ramsay
Gilly,wont long well south soon im excited see oldto...,going oldtown taking,None,None
Jaime,women murdered myrcella overthrown house marte...,None,None,None


### Episode Four

In [74]:
#df_s6.script_text[3]

In [75]:
script_text6_04 = df_s6.script_text[3]
print type(script_text6_04)

#script_text6_04

<type 'list'>


In [76]:
script_split4 = [i.split('\n') for i in script_text6_04]
#script_split4

In [77]:
string_script_text4 = ' '

for i in script_split4:
    for e in i:
        string_script_text4 = string_script_text4 + ' ' + e

In [78]:
#string_script_text4

In [79]:
words4 = word_tokenize(string_script_text4)
#words4

In [80]:
sentences4 = sent_tokenize(string_script_text4)
#sentences4

In [81]:
dialogue4 = []

for sentence in sentences4:
    words=sentence.split(':')[0]
    if len(words.split())<=2:
        if words in (names6_04):
            dialogue4.append(sentence)

In [82]:
#dialogue4

In [83]:
import pickle
with open('/Users/zoemarkovits/Metis/project_five_got/Dialogue_PKLs/s604.pkl', 'wb') as f:
    pickle.dump(dialogue4, f)

In [84]:
from collections import defaultdict

speakers4 = defaultdict(list)
for sentence in dialogue4:
    speakers4[sentence.split(':')[0]].append(sentence.split(':')[1])

#speakers4

In [85]:
speakers4['JON SNOW']

[u' South    DOLOROUS EDD',
 u' Get warm        DOLOROUS EDD',
 u' I did everything I could  You know that    DOLOROUS EDD',
 u' With the peas and onions?',
 u' There\u2019s nothing to forgive    SANSA',
 u' Where will we go?',
 u' I can\u2019t stay here, not after what happened    SANSA',
 u' They didn\u2019t come here to serve me    SANSA',
 u' Sansa    SANSA',
 u' I\u2019m not Lord Commander anymore.',
 u" It's just more of the same.",
 u' Lord of Winterfell and Warden of the North.',
 u' We don\u2019t know that.']

In [86]:
episode_four = pd.DataFrame.from_dict(speakers4, orient="index")
print episode_four.shape

(33, 13)


In [87]:
#from numpy import nan
#episode_four.fillna(value=str(0), inplace=True)

In [88]:
#episode_four.info()

In [89]:
#episode_four.head()

In [90]:
test4 = episode_four[0].apply(remove_stops)
#test4

In [91]:
for column in episode_four.columns:
    for x in range(len(episode_four[column])):
        episode_four[column][x] = remove_stops(episode_four[column][x])

In [92]:
episode_four.index = episode_four.index.str.title()

In [93]:
episode_four.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12
Kevan,king ordered take action high sparrow faith mi...,king call jaime,course want back,doesnt go planned sparrows many friends city,None,None,None,None,None,None,None,None,None
Melisandre,jon snow commands davos,hes prince promised davos,battle stannis defeated davos,None,None,None,None,None,None,None,None,None,None
Jaime,cersei mother king,youll take action,None,None,None,None,None,None,None,None,None,None,None
Tyrion,grey worm,long enough know missandei,instead abolishing slavery overnight give seve...,queen chose advisor returns travels man,soon word man,offered terms man speaking valyrian,None,None,None,None,None,None,None
Littlefinger,lord come see littlefinger,indeed yohn royce,tell lord royce many people knew travel plans ...,lord royce served vale well hes enjoyed distin...,None,None,None,None,None,None,None,None,None


### Episode Five

In [94]:
#df_s6.script_text[4]

In [95]:
script_text6_05 = df_s6.script_text[4]
print type(script_text6_05)

#script_text6_05

<type 'list'>


In [96]:
script_split5 = [i.split('\n') for i in script_text6_05]
#script_split5

In [97]:
string_script_text5 = ' '

for i in script_split5:
    for e in i:
        string_script_text5 = string_script_text5 + ' ' + e

In [98]:
#string_script_text5

In [99]:
words5 = word_tokenize(string_script_text5)
#words5

In [100]:
sentences5 = sent_tokenize(string_script_text5)
#sentences5

In [101]:
dialogue5 = []

for sentence in sentences5:
    words=sentence.split(':')[0]
    if len(words.split())<=2:
        if words in (names6_05):
            dialogue5.append(sentence)

In [102]:
#dialogue5

In [103]:
import pickle
with open('/Users/zoemarkovits/Metis/project_five_got/Dialogue_PKLs/s605.pkl', 'wb') as f:
    pickle.dump(dialogue5, f)

In [105]:
from collections import defaultdict

speakers5 = defaultdict(list)
for sentence in dialogue5:
    speakers5[sentence.split(':')[0]].append(sentence.split(':')[1])

#speakers5

In [106]:
speakers5['AERON']

[u' We speak in the presence of the Drowned God.',
 u' Let the fish eat the scales off his eyes.',
 u' What is dead may never die, but rises again harder and stronger.',
 u' What is dead may never die.',
 u' They stole our best ships.']

In [107]:
episode_five = pd.DataFrame.from_dict(speakers5, orient="index")
print episode_five.shape

(41, 30)


In [108]:
#from numpy import nan
#episode_five.fillna(value=str(0), inplace=True)

In [109]:
#episode_five.info()

In [110]:
#episode_five.head()

In [111]:
test5 = episode_five[0].apply(remove_stops)
#test5

In [112]:
for column in episode_five.columns:
    for x in range(len(episode_five[column])):
        episode_five[column][x] = remove_stops(episode_five[column][x])

In [113]:
episode_five.index = episode_five.index.str.title()

In [114]:
episode_five.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29
Davos,aside starks boltons powerful houses north umb...,umbers karstarks already declared boltons well,beg pardon lady know stark beheaded father,precious little lady,may well loyal many rose boltons betrayed family,dont doubt,didnt know tullys still army,thats good,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
Kinvara,daenerys stormborn one promised,freed slaves chains crucified masters sins,dragons fire made flesh gift lord light,want queen worshipped obeyed,summon eloquent priests,everything lords,everyone reason,tell name one spoke,serve queen,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
Nan,wylis,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
Three-Eyed Raven,touched,knows,,time come,become,,listen friend brandon,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
Tyrion,enough,enough meereen peace,yes know brought security,said anything,thats extent valyrian,thank travelling way,truth need help,yes,indeed,ideally wed avoid purifying many nonbelievers,id settle obeyed,sounds excellent varys,wed grateful support could provide queen,friend healthy skepticism religion loyal suppo...,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None


### Episode Six

In [115]:
#df_s6.script_text[5]

In [116]:
script_text6_06 = df_s6.script_text[5]
print type(script_text6_06)

#script_text6_06

<type 'list'>


In [117]:
script_split6 = [i.split('\n') for i in script_text6_06]
#script_split6

In [118]:
string_script_text6 = ' '

for i in script_split6:
    for e in i:
        string_script_text6 = string_script_text6 + ' ' + e

In [119]:
#string_script_text6

In [120]:
words6 = word_tokenize(string_script_text6)
#words6

In [121]:
sentences6 = sent_tokenize(string_script_text6)
#sentences6

In [122]:
dialogue6 = []

for sentence in sentences6:
    words=sentence.split(':')[0]
    if len(words.split())<=2:
        if words in (names6_06):
            dialogue6.append(sentence)

In [123]:
#dialogue6

In [124]:
import pickle
with open('/Users/zoemarkovits/Metis/project_five_got/Dialogue_PKLs/s606.pkl', 'wb') as f:
    pickle.dump(dialogue6, f)

In [125]:
from collections import defaultdict

speakers6 = defaultdict(list)
for sentence in dialogue6:
    speakers6[sentence.split(':')[0]].append(sentence.split(':')[1])

#speakers6

In [126]:
speakers6['ARYA']

[u' Nothing.',
 u' Three times.',
 u' No.',
 u' You\u2019re very good.',
 u' So change it.',
 u' The queen loves her son more than anything.',
 u' Mercy.',
 u' I have to go.',
 u' Careful of that one.']

In [127]:
episode_six = pd.DataFrame.from_dict(speakers6, orient="index")
print episode_six.shape

(34, 32)


In [128]:
#from numpy import nan
#episode_six.fillna(value=str(0), inplace=True)

In [129]:
#episode_six.info()

In [130]:
#episode_six.head()

In [131]:
test6 = episode_six[0].apply(remove_stops)
#test6

In [132]:
for column in episode_six.columns:
    for x in range(len(episode_six[column])):
        episode_six[column][x] = remove_stops(episode_six[column][x])

In [133]:
episode_six.index = episode_six.index.str.title()

In [134]:
episode_six.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31
Dickon,well,brought yards,much hunting wall,mmhmmm,theres thing,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
Gilly,green,youre nervous,thats better nervous mute,little sam baby,tell family,tell far north,didnt tell im really,think,happy know lady tarly,yes course,hard walk,wield sword,saw eyes way castle black,nights watch came keep,north wall,im angry,youre thinks sam,sam,,dont things,,sisters dress,sam,sam,None,None,None,None,None,None,None,None
Jaime,lord,sorry interrupt,speak king tommen house baratheon first name,already insulted one great house,ive member kingsguard since born,youre sitting chair,walking naked streets,im sent deal blackfish,im going riverlands,im going give bronn largest bag gold anyones e...,son,youll stand trial soon,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
Woman,long live king tommen bless queen margaery,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
Sam,oh yes,didnt think id ever come back father made reno...,certainly didnt expect introducing family,yes thats important,told met north,well wasnt large piece parchment,thing ismy father doesnt like wildlings,oh hello mother,oh,talla,oh um mother gilly,oh son samwell,father,beautiful,oh course,oh thats fair distance,oh yes,rabbits,oh yes,travelled south orders lord commander,yes,gilly,first light,say goodbye,goodbye gilly,leaving,get things,belong together,oh theyll understand,wont mind,familys sword,bloody well try


### Episode Seven

In [135]:
#df_s6.script_text[6]

In [136]:
script_text6_07 = df_s6.script_text[6]
print type(script_text6_07)

#script_text6_07

<type 'list'>


In [137]:
script_split7 = [i.split('\n') for i in script_text6_07]
#script_split7

In [138]:
string_script_text7 = ' '

for i in script_split7:
    for e in i:
        string_script_text7 = string_script_text7 + ' ' + e

In [139]:
#string_script_text7

In [140]:
words7 = word_tokenize(string_script_text7)
#words7

In [141]:
sentences7 = sent_tokenize(string_script_text7)
#sentences7

In [142]:
dialogue7 = []

for sentence in sentences7:
    words=sentence.split(':')[0]
    if len(words.split())<=2:
        if words in (names6_07):
            dialogue7.append(sentence)

In [143]:
#dialogue7

In [144]:
import pickle
with open('/Users/zoemarkovits/Metis/project_five_got/Dialogue_PKLs/s607.pkl', 'wb') as f:
    pickle.dump(dialogue7, f)

In [145]:
from collections import defaultdict

speakers7 = defaultdict(list)
for sentence in dialogue7:
    speakers7[sentence.split(':')[0]].append(sentence.split(':')[1])

#speakers7

In [146]:
speakers7['BRONN']

[u' Now, that is a sorry attempt at a siege.',
 u' Oh, no.',
 u' Aye, knights don\u2019t get paid.',
 u' That\u2019s like saying I have a bigger cock than anyone in the Unsullied army.',
 u' You promised me a lordship and a castle and a highborn beauty for a wife.',
 u' Good thing we\u2019re friends or we\u2019d be fucking you in the ass right now.',
 u' Have your lads start digging perimeter trenches.',
 u' A parley or a fight?',
 u' You\u2019ve got one hand.']

In [147]:
episode_seven = pd.DataFrame.from_dict(speakers7, orient="index")
print episode_seven.shape

(26, 31)


In [148]:
#from numpy import nan
#episode_seven.fillna(value=str(0), inplace=True)

In [149]:
#episode_seven.info()

In [150]:
#episode_seven.head()

In [151]:
test7 = episode_seven[0].apply(remove_stops)
#test7

In [152]:
for column in episode_seven.columns:
    for x in range(len(episode_seven[column])):
        episode_seven[column][x] = remove_stops(episode_seven[column][x])

In [153]:
episode_seven.index = episode_seven.index.str.title()

In [154]:
episode_seven.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30
Davos,please lady understand feel,davos lady house seaworth,never thought youd find position,uncle lord commander mormont made man steward,long boltons hold winterfell north divided,theyre half ferocious lady boltons doomed,stannis camped way winterfell,experienced commander westeros,wildlings,wed hoped,fucks sake,friends worth fighting,thisll,hold,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
Bronn,sorry attempt siege,oh,aye knights dont get paid,thats like saying bigger cock anyone unsullied...,promised lordship castle highborn beauty wife,good thing friends wed fucking ass right,lads start digging perimeter trenches,parley fight,youve got one hand,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
Jaime,someone certainly,youre anointed knight,better instincts officer lannister army,expect command lannister forces long,youll get three,lothar,cause didnt set proper perimeter,lord edmure bathed fed,fool makes threats hes prepared carry,siege command,get word blackfish,hes old man,blackfish,dont,know im,wont stop edmure,well breach kill every last one,war ser,clearly intention saving mens lives,well,None,None,None,None,None,None,None,None,None,None,None
Brynden,go,kingslayer,assume youre fulfill vow gave niece,pity,,nephews marked death matter,hundreds mine,honor,long im standing war,sieges dull,aye,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
Yara,us still like,dont go far love,ill find bit,nothing iron islands ass like,im sorry,course,great big world fast ships,dont care want,drink,youre ironborn theon,im tired watching cower like beat dog,listen,escaped hear,fuck justice,listen,,since last night ashore long im gonna go fuck ...,None,None,None,None,None,None,None,None,None,None,None,None,None,None


### Episode Eight

In [155]:
#df_s6.script_text[7]

In [156]:
script_text6_08 = df_s6.script_text[7]
print type(script_text6_08)

#script_text6_08

<type 'list'>


In [157]:
script_split8 = [i.split('\n') for i in script_text6_08]
#script_split8

In [158]:
string_script_text8 = ' '

for i in script_split8:
    for e in i:
        string_script_text8 = string_script_text8 + ' ' + e

In [159]:
#string_script_text8

In [160]:
words8 = word_tokenize(string_script_text8)
#words8

In [161]:
sentences8 = sent_tokenize(string_script_text8)
#sentences8

In [162]:
dialogue8 = []

for sentence in sentences8:
    words=sentence.split(':')[0]
    if len(words.split())<=2:
        if words in (names6_08):
            dialogue8.append(sentence)

In [163]:
#dialogue8

In [164]:
import pickle
with open('/Users/zoemarkovits/Metis/project_five_got/Dialogue_PKLs/s608.pkl', 'wb') as f:
    pickle.dump(dialogue8, f)

In [165]:
from collections import defaultdict

speakers8 = defaultdict(list)
for sentence in dialogue8:
    speakers8[sentence.split(':')[0]].append(sentence.split(':')[1])

#speakers8

In [166]:
speakers8['ARYA']

[u' And good at putting holes in them.',
 u' What happened to the actress?',
 u' Mmm.',
 u' I can\u2019t.',
 u' I don\u2019t think I could remember all of the lines.',
 u' You wouldn\u2019t be safe.',
 u' She doesn\u2019t have a name.',
 u' What\u2019s that?',
 u' I don\u2019t want any.',
 u' Lady Crane?',
 u' Lady Crane?',
 u' A girl is Arya Stark of Winterfell and I\u2019m going home.']

In [167]:
episode_eight = pd.DataFrame.from_dict(speakers8, orient="index")
print episode_eight.shape

(29, 36)


In [168]:
#from numpy import nan
#episode_eight.fillna(value=str(0), inplace=True)

In [169]:
#episode_eight.info()

In [170]:
#episode_eight.head()

In [171]:
test8 = episode_eight[0].apply(remove_stops)
#test8

In [172]:
for column in episode_eight.columns:
    for x in range(len(episode_eight[column])):
        episode_eight[column][x] = remove_stops(episode_eight[column][x])

In [173]:
episode_eight.index = episode_eight.index.str.title()

In [174]:
episode_eight.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35
Kevan,,royal announcement throne room moment,going,place gallery ladies court,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
Bronn,podrick fucking payne,,think theyre fucking,,oh,must shown,,come old cunt like still sneak murder,thats truth isnt,right lets start footwork,show stance,listen pod,lesson number one assume everyone wants hit,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
Jaime,experience girls like dont live long,well im proud,youre welcome,army plan taking winterfell,theyre bit occupied moment,riverrun granted freys royal decree,exactly,shouldnt argue politics,im lannister,propose,would abandon ancestral home,ever met blackfish,hes even stubborn,right,word,,yes lady brienne,course,lets hope doesnt come,lord edmure,apologize way freys treated,blackfish old man,,ill remind houses war,give impression negotiation,ill leave judgments gods,sisters prisoner,perhaps didnt,dont im telling anyway youre prisoner,im trade insults,ill talk whomever want,loved children,theyd anything protect babies,thats exactly came,ill launch riverrun catapult,good
Tyrion,id call successful gambit,look around,worked,spoken like man never shave,im going miss,hope youre right expedition,need queen,varys,famous dwarf world,happy,like man reborn,dont drink,,made rules,miserable old shits didnt want human,thats know working,heres queen,daenerys stormborn mother dragons breaker chai...,like,yes yes,tell joke missandei naath,grey worm,right,yes,story thats supposed make laugh ideally,missandei like wine,tell joke,lied us,cut,unsullied could mount defense beachhead,acknowledged,right army,,None,None,None
Missandei,seem happy,happy queen returns,tried wine,know jokes,joke like story torgo nudho,,two translators sinking ship,first says know swim second says shout help la...,dont even know joke,jokes,changes nothing,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None


### Episode Nine

In [175]:
#df_s6.script_text[8]

In [176]:
script_text6_09 = df_s6.script_text[8]
print type(script_text6_09)

#script_text6_09

<type 'list'>


In [177]:
script_split9 = [i.split('\n') for i in script_text6_09]
#script_split9

In [178]:
string_script_text9 = ' '

for i in script_split9:
    for e in i:
        string_script_text9 = string_script_text9 + ' ' + e

In [179]:
#string_script_text9

In [180]:
words9 = word_tokenize(string_script_text9)
#words9

In [181]:
sentences9 = sent_tokenize(string_script_text9)
#sentences9

In [182]:
dialogue9 = []

for sentence in sentences9:
    words=sentence.split(':')[0]
    if len(words.split())<=2:
        if words in (names6_09):
            dialogue9.append(sentence)

In [183]:
#dialogue9

In [184]:
import pickle
with open('/Users/zoemarkovits/Metis/project_five_got/Dialogue_PKLs/s609.pkl', 'wb') as f:
    pickle.dump(dialogue9, f)

In [185]:
from collections import defaultdict

speakers9 = defaultdict(list)
for sentence in dialogue9:
    speakers9[sentence.split(':')[0]].append(sentence.split(':')[1])

#speakers9

In [186]:
speakers9['DAVOS']

[u' That\u2019s not his way.',
 u' It\u2019s crucial that we let them charge at us.',
 u' We should all get some sleep.',
 u' It wasn\u2019t the Boltons defeated Stannis.',
 u' What?',
 u' No, it\u2019s a manner of speaking.',
 u' Maybe that was our mistake, believing in kings.',
 u' No, he\u2019s not.',
 u' No, thanks.',
 u' I walk.',
 u' Prepare to charge!',
 u' Go!',
 u' We\u2019ll just kill our own men.',
 u' Follow!',
 u' With me, lads!']

In [187]:
episode_nine = pd.DataFrame.from_dict(speakers9, orient="index")
print episode_nine.shape

(19, 28)


In [188]:
#from numpy import nan
#episode_nine.fillna(value=str(0), inplace=True)

In [189]:
#episode_nine.info()

In [190]:
#episode_nine.head()

In [191]:
test9 = episode_nine[0].apply(remove_stops)
#test9

In [192]:
for column in episode_nine.columns:
    for x in range(len(episode_nine[column])):
        episode_nine[column][x] = remove_stops(episode_nine[column][x])

In [193]:
episode_nine.index = episode_nine.index.str.title()

In [194]:
episode_nine.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27
Yara,paid,westeros,common,never demand im anything really,wed like help us murder uncle two dont think w...,thats way life,,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
Davos,thats way,crucial let charge us,get sleep,wasnt boltons defeated stannis,,manner speaking,maybe mistake believing kings,hes,thanks,walk,prepare charge,go,well kill men,follow,lads,None,None,None,None,None,None,None,None,None,None,None,None,None
Karstark,cavalry charge,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
Sansa,yes,know,youve met enemy drawn battle plans,youve known space single conversation trusted ...,think hes going fall trap,plays people,dont know,well never get back,listen please,dont know,think thats obvious,asked advice earlier would told attack winterf...,enough,ramsay wins im going back alive,one protect,jon,,words disappear,havent fed seven days,,None,None,None,None,None,None,None,None
Man #2,get back,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None


### Episode Ten

In [195]:
#df_s6.script_text[9]

In [196]:
script_text6_10 = df_s6.script_text[9]
print type(script_text6_10)

#script_text6_10

<type 'list'>


In [197]:
script_split10 = [i.split('\n') for i in script_text6_10]
#script_split10

In [198]:
string_script_text10 = ' '

for i in script_split10:
    for e in i:
        string_script_text10 = string_script_text10 + ' ' + e

In [199]:
#string_script_text10

In [200]:
words10 = word_tokenize(string_script_text10)
#words10

In [201]:
sentences10 = sent_tokenize(string_script_text10)
#sentences10

In [202]:
dialogue10 = []

for sentence in sentences10:
    words=sentence.split(':')[0]
    if len(words.split())<=2:
        if words in (names6_10):
            dialogue10.append(sentence)

In [203]:
#dialogue10

In [204]:
import pickle
with open('/Users/zoemarkovits/Metis/project_five_got/Dialogue_PKLs/s610.pkl', 'wb') as f:
    pickle.dump(dialogue10, f)

In [205]:
from collections import defaultdict

speakers10 = defaultdict(list)
for sentence in dialogue10:
    speakers10[sentence.split(':')[0]].append(sentence.split(':')[1])

#speakers10

In [206]:
speakers10['CERSEI']

[u' Confess.',
 u' Confess.',
 u' You\u2019ve always been quiet.',
 u' I said my face would be the last thing you saw before you died.',
 u' What?',
 u' This is Ser Gregor Clegane.',
 u' Your gods have forsaken you.',
 u' Shame.',
 u' Shame.',
 u' Shame.',
 u' He should be with his grandfather, his brother, his sister.']

In [207]:
episode_ten = pd.DataFrame.from_dict(speakers10, orient="index")
print episode_ten.shape

(42, 20)


In [208]:
#from numpy import nan
#episode_ten.fillna(value=str(0), inplace=True)

In [209]:
#episode_ten.info()

In [210]:
#episode_ten.head()

In [211]:
test10 = episode_ten[0].apply(remove_stops)
#test10

In [212]:
for column in episode_ten.columns:
    for x in range(len(episode_ten[column])):
        episode_ten[column][x] = remove_stops(episode_ten[column][x])

In [213]:
episode_ten.index = episode_ten.index.str.title()

In [214]:
episode_ten.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
Davos,tell,tell,,burned little girl alive,commands burn children lord evil,loved girl like,way,aye wrong,ever come back way execute,None,None,None,None,None,None,None,None,None,None,None
Septa Unella,,good,,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
Pycelle,later,whats meaning,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
Ned,lyanna,im dream,ive missed,shh,youre going die,get water,maester,None,None,None,None,None,None,None,None,None,None,None,None,None
Melisandre,could worse jon snow,princess shireen,burned stake,army trapped,lord commands,standing,father,didnt lie,ive ready die many years,None,None,None,None,None,None,None,None,None,None,None


## Season Six

In [215]:
episode_one.index

Index([u'Davos', u'Male Voice #2', u'Jaime', u'Tyrion', u'Wife #1',
       u'Dolorous Edd', u'Red Priest', u'Maester Wolkan', u'Daenerys',
       u'Doran', u'Roose', u'Maester', u'Theon', u'Cersei', u'Varys',
       u'Trystane', u'Margaery', u'Ramsay', u'Nymeria', u'Obara',
       u'Alliser Thorne', u'Jorah', u'High Sparrow', u'Bloodrider #2'],
      dtype='object')

In [216]:
episode_two.index

Index([u'Young Benjen', u'Theon', u'Jaime', u'Roose', u'Tommen', u'Sansa',
       u'Nan', u'Waif', u'Bran', u'Hodor', u'Melisandre', u'Hooded Figure',
       u'Tyrion', u'High Sparrow', u'Balon', u'Arya', u'Missandei',
       u'Alliser Thorne', u'Ramsay'],
      dtype='object')

In [217]:
print episode_one.shape
print episode_two.shape

(24, 6)
(19, 4)


In [218]:
season_six = episode_one.join(episode_two, rsuffix="_two", how='outer')
print season_six.shape

(36, 10)


In [219]:
season_six.index

Index([u'Alliser Thorne', u'Arya', u'Balon', u'Bloodrider #2', u'Bran',
       u'Cersei', u'Daenerys', u'Davos', u'Dolorous Edd', u'Doran',
       u'High Sparrow', u'Hodor', u'Hooded Figure', u'Jaime', u'Jorah',
       u'Maester', u'Maester Wolkan', u'Male Voice #2', u'Margaery',
       u'Melisandre', u'Missandei', u'Nan', u'Nymeria', u'Obara', u'Ramsay',
       u'Red Priest', u'Roose', u'Sansa', u'Theon', u'Tommen', u'Trystane',
       u'Tyrion', u'Varys', u'Waif', u'Wife #1', u'Young Benjen'],
      dtype='object')

In [220]:
episode_three.index

Index([u'Davos', u'Melisandre', u'Smalljon', u'Gilly', u'Jaime',
       u'Three-Eyed Raven', u'Tyrion', u'Missandei', u'Dolorous Edd',
       u'Samwell', u'Arthur', u'Grey Worm', u'Cersei', u'Varys', u'Qyburn',
       u'Jon Snow', u'Ramsay', u'Little Bird', u'Bran', u'High Sparrow',
       u'Arya', u'Maester Pycelle'],
      dtype='object')

In [221]:
episode_three.shape

(22, 4)

In [222]:
season_six = season_six.join(episode_three, rsuffix="_three", how='outer')
print season_six.shape

(46, 14)


In [223]:
#season_six.index

In [224]:
episode_four.index

Index([u'Kevan', u'Melisandre', u'Jaime', u'Tyrion', u'Littlefinger', u'Robin',
       u'Yara', u'Maester Pycelle', u'Yohn Royce', u'Khal #3', u'Daenerys',
       u'Lady Olenna', u'Theon', u'Tommen', u'Cersei', u'Osha', u'Margaery',
       u'Jon Snow', u'Ramsay', u'Khal #4', u'Lhazareen Woman', u'Khal #1',
       u'Khal #2', u'Sansa', u'Brienne', u'Khal Moro', u'Jorah',
       u'Dothraki #1', u'Daario', u'Tormund', u'High Sparrow', u'Loras',
       u'Man'],
      dtype='object')

In [225]:
episode_four.shape

(33, 13)

In [226]:
season_six = season_six.join(episode_four, rsuffix="_four", how='outer')
print season_six.shape

(65, 27)


In [227]:
#season_six.index

In [228]:
episode_five.index

Index([u'Davos', u'Kinvara', u'Nan', u'Three-Eyed Raven', u'Tyrion', u'Euron',
       u'Littlefinger', u'Missandei', u'Dolorous Edd', u'Yara', u'Young Hodor',
       u'Aerson', u'Theon', u'Lady Crane', u'Rickard', u'Hodor', u'Meera',
       u'Daenerys', u'Clarenzo', u'Ladyc Rane', u'Grey Worm', u'Waif',
       u'Varys', u'Bobono', u'Bianca', u'Izembaro', u'Jon Snow', u'All',
       u'Leaf', u'Aeron', u'Sansa', u'Brienne', u'Man #2', u'Man #1', u'Jorah',
       u'Jaqen', u'Camello', u'Young Ned', u'Arya', u'Bran', u'Man'],
      dtype='object')

In [229]:
episode_five = episode_five.reset_index()

In [230]:
episode_five_dict = {'Davos':'Davos', 
                     'Kinvara':'Kinvara', 
                     'Nan':'Nan', 
                     'Three-Eyed Raven':'Three-Eyed Raven', 
                     'Tyrion':'Tyrion', 
                     'Euron':'Euron',
                     'Littlefinger':'Littlefinger', 
                     'Missandei':'Missandei', 
                     'Dolorous Edd':'Dolorous Edd', 
                     'Yara':'Yara', 
                     'Young Hodor':'Young Hodor',
                     'Aerson':'Aeron', 
                     'Theon':'Theon', 
                     'Lady Crane':'Lady Crane', 
                     'Rickard':'Rickard', 
                     'Hodor':'Hodor', 
                     'Meera':'Meera',
                     'Daenerys':'Daenerys', 
                     'Clarenzo':'Clarenzo', 
                     'Ladyc Rane':'Lady Crane', 
                     'Grey Worm':'Grey Worm', 
                     'Waif':'Waif',
                     'Varys':'Varys', 
                     'Bobono':'Bobono', 
                     'Bianca':'Bianca', 
                     'Izembaro':'Izembaro', 
                     'Jon Snow':'Jon Snow', 
                     'All':'All',
                     'Leaf':'Leaf', 
                     'Aeron':'Aeron', 
                     'Sansa':'Sansa', 
                     'Brienne':'Brienne', 
                     'Man #2':'Man #2', 
                     'Man #1':'Man #1', 
                     'Jorah':'Jorah',
                     'Jaqen':'Jaqen', 
                     'Camello':'Camello', 
                     'Young Ned':'Young Ned', 
                     'Arya':'Arya', 
                     'Bran':'Bran', 
                     'Man':'Man'}

episode_five['index'] = episode_five['index'].replace(episode_five_dict)

In [231]:
episode_five = episode_five.set_index('index')

In [232]:
episode_five.shape

(41, 30)

In [233]:
season_six = season_six.join(episode_five, rsuffix="_five", how='outer')
print season_six.shape    

(85, 57)


In [234]:
#season_six.index

In [235]:
episode_six.index

Index([u'Dickon', u'Gilly', u'Jaime', u'Woman', u'Sam', u'Olenna',
       u'Lady Crane', u'Talla', u'Meera', u'Lothar', u'Daenerys', u'Benjen',
       u'Cersei', u'Clarenzo', u'Tommen', u'Waif', u'Bobono', u'Bianca',
       u'Mace', u'Izembaro', u'Melessa', u'Margaery', u'Walder', u'Rider',
       u'Randyll', u'All', u'Captain', u'Jaqen', u'Daario', u'Camello',
       u'High Sparrow', u'Black Walder', u'Arya', u'Bran'],
      dtype='object')

In [236]:
episode_six.shape

(34, 32)

In [237]:
season_six = season_six.join(episode_six, rsuffix="_six", how='outer')
print season_six.shape    

(99, 89)


In [238]:
#season_six.index

In [239]:
episode_seven.index

Index([u'Davos', u'Bronn', u'Jaime', u'Brynden', u'Yara', u'Woman', u'Olenna',
       u'Lothar', u'Ray', u'Theon', u'Cersei', u'Dim Dalba', u'Sandor',
       u'Robett', u'Lyanna', u'Margaery', u'Jon Snow', u'Sansa', u'Man #2',
       u'Lem', u'Wun Wun', u'Tormund', u'High Sparrow', u'Black Walder',
       u'Arya', u'Man'],
      dtype='object')

In [240]:
episode_seven.shape

(26, 31)

In [241]:
season_six = season_six.join(episode_seven, rsuffix="_seven", how='outer')
print season_six.shape

(108, 120)


In [242]:
episode_eight.index

Index([u'Kevan', u'Bronn', u'Jaime', u'Tyrion', u'Missandei', u'Crowd',
       u'Lady Crane', u'Guard', u'Guard Captain', u'Thoros', u'Gatins',
       u'Tommen', u'Grey Worm', u'Young Man', u'Varys', u'Cersei', u'Sandor',
       u'Beric', u'Brynden', u'Qyburn', u'Edmure', u'Podrick', u'Lancel',
       u'Brienne', u'Lem', u'Jaqen', u'Waif', u'Arya', u'Man'],
      dtype='object')

In [243]:
episode_eight.shape

(29, 36)

In [244]:
season_six = season_six.join(episode_eight, rsuffix="_eight", how='outer')
print season_six.shape

(118, 156)


In [245]:
episode_nine.index

Index([u'Yara', u'Davos', u'Karstark', u'Sansa', u'Man #2', u'Tyrion',
       u'Melisandre', u'All', u'Smalljon', u'Tormund', u'Razdal', u'Yezzan',
       u'Daenerys', u'Belicho', u'Theon', u'Man', u'Missandei', u'Jon Snow',
       u'Ramsay'],
      dtype='object')

In [246]:
episode_nine.shape

(19, 28)

In [247]:
season_six = season_six.join(episode_nine, rsuffix="_nine", how='outer')
print season_six.shape

(122, 184)


In [248]:
episode_ten.index

Index([u'Davos', u'Septa Unella', u'Pycelle', u'Ned', u'Melisandre', u'Bronn',
       u'Jaime', u'Tyrion', u'Girl', u'Maester', u'Ellaria', u'Sam', u'Olenna',
       u'Meera', u'Daenerys', u'Benjen', u'Vale Knight', u'Tommen', u'Cersei',
       u'Varys', u'Walder', u'Handmaiden', u'Mace', u'Lyanna', u'Margaery',
       u'Qyburn', u'Jon Snow', u'Nymeria', u'All', u'Lancel', u'Obara',
       u'Sansa', u'Men', u'Manderly', u'Daario', u'Tormund', u'Glover',
       u'High Sparrow', u'Man', u'Bran', u'Loras', u'Petyr'],
      dtype='object')

In [249]:
episode_ten.shape

(42, 20)

In [250]:
season_six = season_six.join(episode_ten, rsuffix="_ten", how='outer')
print season_six.shape

(133, 204)


In [251]:
#season_six.index

In [252]:
from numpy import nan
season_six.fillna(value=str(0), inplace=True)

In [253]:
#season_six.head(135)

In [254]:
season_six.info()

<class 'pandas.core.frame.DataFrame'>
Index: 133 entries, Aeron to Young Ned
Columns: 204 entries, 0 to 19_ten
dtypes: object(204)
memory usage: 213.0+ KB


In [255]:
season_six.to_pickle('/Users/zoemarkovits/Metis/project_five_got/PKLs/season_six.pkl')

## Character Breakdown

### Arya Stark

In [256]:
Arya = season_six.loc['Arya']
#print Arya

In [257]:
Arya_Text = Arya.values.tolist()
#Arya_Text

In [258]:
Arya_Words = []

for word in Arya_Text:
    if len(word)>=1:
        if word != '0':
            Arya_Words.append(word)
        
#Arya_Words 

In [259]:
Arya_Words_String = ' '.join(Arya_Words)
#Arya_Words_String

In [260]:
Arya_String_Length = len(Arya_Words_String)
Arya_String_Length

487

In [261]:
Arya_Sentiment = TextBlob(Arya_Words_String).sentiment
Arya_Sentiment

Sentiment(polarity=0.3018518518518518, subjectivity=0.5444444444444445)

### Brienne of Tarth

In [262]:
Brienne = season_six.loc['Brienne']
#print Brienne

In [263]:
Brienne_Text = Brienne.values.tolist()
#Brienne_Text

In [264]:
Brienne_Words = []

for word in Brienne_Text:
    if len(word)>=1:
        if word != '0':
            Brienne_Words.append(word)
        
#Brienne_Words 

In [265]:
Brienne_Words_String = ' '.join(Brienne_Words)
#Brienne_Words_String

In [266]:
Brienne_String_Length = len(Brienne_Words_String)
Brienne_String_Length

909

In [267]:
Brienne_Sentiment = TextBlob(Brienne_Words_String).sentiment
Brienne_Sentiment

Sentiment(polarity=-0.049090909090909095, subjectivity=0.3733333333333333)

### Cersei Lannister

In [268]:
Cersei = season_six.loc['Cersei']
#print Cersei

In [269]:
Cersei_Text = Cersei.values.tolist()
#Cersei_Text

In [270]:
Cersei_Words = []

for word in Cersei_Text:
    if len(word)>=1:
        if word != '0':
            Cersei_Words.append(word)
        
#Cersei_Words 

In [271]:
Cersei_Words_String = ' '.join(Cersei_Words)
#Cersei_Words_String

In [272]:
Cersei_String_Length = len(Cersei_Words_String)
Cersei_String_Length

1061

In [273]:
Cersei_Sentiment = TextBlob(Cersei_Words_String).sentiment
Cersei_Sentiment

Sentiment(polarity=0.10739047619047618, subjectivity=0.44899047619047616)

### Daenerys Targaryen

In [274]:
Daenerys = season_six.loc['Daenerys']
#print Daenerys

In [275]:
Dany_Text = Daenerys.values.tolist()
#Dany_Text

In [276]:
Dany_Words = []

for word in Dany_Text:
    if len(word)>=1:
        if word != '0':
            Dany_Words.append(word)
        
#Dany_Words 

In [277]:
Dany_Words_String = ' '.join(Dany_Words)
#Dany_Words_String

In [278]:
Dany_String_Length = len(Dany_Words_String)
Dany_String_Length

1272

In [279]:
Dany_Sentiment = TextBlob(Dany_Words_String).sentiment
Dany_Sentiment

Sentiment(polarity=0.15000000000000002, subjectivity=0.6253968253968254)

### Davos

In [280]:
Davos = season_six.loc['Davos']
#print Davos

In [281]:
Davos_Text = Davos.values.tolist()
#Davos_Text

In [282]:
Davos_Words = []

for word in Davos_Text:
    if len(word)>=1:
        if word != '0':
            Davos_Words.append(word)
        
#Davos_Words 

In [283]:
Davos_Words_String = ' '.join(Davos_Words)
#Davos_Words_String

In [284]:
Davos_String_Length = len(Davos_Words_String)
Davos_String_Length

1234

In [285]:
Davos_Sentiment = TextBlob(Davos_Words_String).sentiment
Davos_Sentiment

Sentiment(polarity=0.0673611111111111, subjectivity=0.6229166666666667)

### Gilly

In [286]:
Gilly = season_six.loc['Gilly']
#print Gilly

In [287]:
Gilly_Text = Gilly.values.tolist()
#Gilly_Text

In [288]:
Gilly_Words = []

for word in Gilly_Text:
    if len(word)>=1:
        if word != '0':
            Gilly_Words.append(word)
        
#Gilly_Words 

In [289]:
Gilly_Words_String = ' '.join(Gilly_Words)
#Gilly_Words_String

In [290]:
Gilly_String_Length = len(Gilly_Words_String)
Gilly_String_Length

413

In [291]:
Gilly_Sentiment = TextBlob(Gilly_Words_String).sentiment
Gilly_Sentiment

Sentiment(polarity=0.11909722222222224, subjectivity=0.6354166666666666)

### Jaime Lannister

In [292]:
Jaime = season_six.loc['Jaime']
#print Jaime

In [293]:
Jaime_Text = Jaime.values.tolist()
#Jaime_Text

In [294]:
Jaime_Words = []

for word in Jaime_Text:
    if len(word)>=1:
        if word != '0':
            Jaime_Words.append(word)
        
#Jaime_Words 

In [295]:
Jaime_Words_String = ' '.join(Jaime_Words)
#Jaime_Words_String

In [296]:
Jaime_String_Length = len(Jaime_Words_String)
Jaime_String_Length

1999

In [297]:
Jaime_Sentiment = TextBlob(Jaime_Words_String).sentiment
Jaime_Sentiment

Sentiment(polarity=0.15543062200956936, subjectivity=0.5173517126148706)

### Jon Snow

In [298]:
Jon = season_six.loc['Jon Snow']
#print Jon

In [299]:
Jon_Text = Jon.values.tolist()
#Jon_Text

In [300]:
Jon_Words = []

for word in Jon_Text:
    if len(word)>=1:
        if word != '0':
            Jon_Words.append(word)
        
#Jon_Words 

In [301]:
Jon_Words_String = ' '.join(Jon_Words)
#Jon_Words_String

In [302]:
Jon_String_Length = len(Jon_Words_String)
Jon_String_Length

1603

In [303]:
Jon_Sentiment = TextBlob(Jon_Words_String).sentiment
Jon_Sentiment

Sentiment(polarity=0.18095238095238095, subjectivity=0.4542162698412699)

### Jorah Mormont

In [304]:
Jorah = season_six.loc['Jorah']
#print Jorah

In [305]:
Jorah_Text = Jorah.values.tolist()
#Jorah_Text

In [306]:
Jorah_Words = []

for word in Jorah_Text:
    if len(word)>=1:
        if word != '0':
            Jorah_Words.append(word)
        
#Jorah_Words 

In [307]:
Jorah_Words_String = ' '.join(Jorah_Words)
#Jorah_Words_String

In [308]:
Jorah_String_Length = len(Jorah_Words_String)
Jorah_String_Length

423

In [309]:
Jorah_Sentiment = TextBlob(Jorah_Words_String).sentiment
Jorah_Sentiment

Sentiment(polarity=0.029166666666666674, subjectivity=0.475)

### Petyr Baelish/Littlefinger

In [314]:
Littlefinger = season_six.loc['Littlefinger']
Petyr = season_six.loc['Petyr']

In [315]:
Littlefinger_Text = Littlefinger.values.tolist()
Petyr_Text = Petyr.values.tolist()

In [318]:
Littlefinger_Words = []

for word in Littlefinger_Text:
    if len(word)>=1:
        if word != '0':
            Littlefinger_Words.append(word)

for word in Petyr_Text:
    if len(word)>=1:
        if word != '0':
            Littlefinger_Words.append(word)            
            
#Littlefinger_Words

In [319]:
Littlefinger_Words_String = ' '.join(Littlefinger_Words)
#Littlefinger_Words_String

In [320]:
Littlefinger_String_Length = len(Littlefinger_Words_String)
Littlefinger_String_Length

549

In [321]:
Littlefinger_Sentiment = TextBlob(Littlefinger_Words_String).sentiment
Littlefinger_Sentiment

Sentiment(polarity=0.03484848484848485, subjectivity=0.5954545454545456)

### Margaery Tyrell

In [322]:
Margaery = season_six.loc['Margaery']
#print Margaery

In [323]:
Margaery_Text = Margaery.values.tolist()
#Margaery_Text

In [324]:
Margaery_Words = []

for word in Margaery_Text:
    if len(word)>=1:
        if word != '0':
            Margaery_Words.append(word)
        
#Margaery_Words 

In [325]:
Margaery_Words_String = ' '.join(Margaery_Words)
#Margaery_Words_String

In [326]:
Margaery_String_Length = len(Margaery_Words_String)
Margaery_String_Length

662

In [327]:
Margaery_Sentiment = TextBlob(Margaery_Words_String).sentiment
Margaery_Sentiment

Sentiment(polarity=0.21913284632034633, subjectivity=0.6314781746031745)

### Melisandre

In [328]:
Melisandre = season_six.loc['Melisandre']
#print Melisandre

In [329]:
Melisandre_Text = Melisandre.values.tolist()
#Melisandre_Text

In [330]:
Melisandre_Words = []

for word in Melisandre_Text:
    if len(word)>=1:
        if word != '0':
            Melisandre_Words.append(word)
        
#Melisandre_Words 

In [331]:
Melisandre_Words_String = ' '.join(Melisandre_Words)
#Melisandre_Words_String

In [332]:
Melisandre_String_Length = len(Melisandre_Words_String)
Melisandre_String_Length

425

In [333]:
Melisandre_Sentiment = TextBlob(Melisandre_Words_String).sentiment
Melisandre_Sentiment

Sentiment(polarity=7.93016446160826e-18, subjectivity=0.4714285714285714)

### Missandei

In [334]:
Missandei = season_six.loc['Missandei']
#print Missandei

In [326]:
type(Missandei)

pandas.core.series.Series

In [335]:
Missandei_Text = Missandei.values.tolist()
#Missandei_Text

In [336]:
Missandei_Words = []

for word in Missandei_Text:
    if len(word)>=1:
        if word != '0':
            Missandei_Words.append(word)
        
#Missandei_Words 

In [337]:
Missandei_Words_String = ' '.join(Missandei_Words)
#Missandei_Words_String

In [338]:
Missandei_String_Length = len(Missandei_Words_String)
Missandei_String_Length

457

In [339]:
Missandei_Sentiment = TextBlob(Missandei_Words_String).sentiment
Missandei_Sentiment

Sentiment(polarity=0.3722222222222222, subjectivity=0.43333333333333335)

### Olenna Tyrell

In [340]:
Olenna1 = season_six.loc['Lady Olenna']
Olenna2 = season_six.loc['Olenna']

In [341]:
Olenna_Text1 = Olenna1.values.tolist()
Olenna_Text2 = Olenna2.values.tolist()

In [342]:
Olenna_Words = []

for word in Olenna_Text1:
    if len(word)>=1:
        if word != '0':
            Olenna_Words.append(word)

for word in Olenna_Text2:
    if len(word)>=1:
        if word != '0':
            Olenna_Words.append(word)            
            
#Olenna_Words 

In [343]:
Olenna_Words_String = ' '.join(Olenna_Words)
#Olenna_Words_String

In [344]:
Olenna_String_Length = len(Olenna_Words_String)
Olenna_String_Length

554

In [345]:
Olenna_Sentiment = TextBlob(Olenna_Words_String).sentiment
Olenna_Sentiment

Sentiment(polarity=-0.0361111111111111, subjectivity=0.4694444444444444)

### Ramsay Bolton

In [346]:
Ramsay = season_six.loc['Ramsay']
#print Ramsay

In [347]:
Ramsay_Text = Ramsay.values.tolist()
#Ramsay_Text

In [348]:
Ramsay_Words = []

for word in Ramsay_Text:
    if len(word)>=1:
        if word != '0':
            Ramsay_Words.append(word)
        
#Ramsay_Words 

In [349]:
Ramsay_Words_String = ' '.join(Ramsay_Words)
#Ramsay_Words_String

In [350]:
Ramsay_String_Length = len(Ramsay_Words_String)
Ramsay_String_Length

634

In [351]:
Ramsay_Sentiment = TextBlob(Ramsay_Words_String).sentiment
Ramsay_Sentiment

Sentiment(polarity=0.308482905982906, subjectivity=0.6155413105413107)

### Samwell Tarly

In [352]:
Sam1 = season_six.loc['Sam']
Sam2 = season_six.loc['Samwell']

In [353]:
Sam_Text1 = Sam1.values.tolist()
Sam_Text2 = Sam2.values.tolist()

In [355]:
Sam_Words = []

for word in Sam_Text1:
    if len(word)>=1:
        if word != '0':
            Sam_Words.append(word)

for word in Sam_Text2:
    if len(word)>=1:
        if word != '0':
            Sam_Words.append(word)            
            
#Sam_Words 

In [356]:
Sam_Words_String = ' '.join(Sam_Words)
#Sam_Words_String

In [357]:
Sam_String_Length = len(Sam_Words_String)
Sam_String_Length

885

In [358]:
Sam_Sentiment = TextBlob(Sam_Words_String).sentiment
Sam_Sentiment

Sentiment(polarity=0.1780844155844156, subjectivity=0.6277183600713013)

### Sansa Stark

In [359]:
Sansa = season_six.loc['Sansa']
#print Sansa

In [360]:
Sansa_Text = Sansa.values.tolist()
#Sansa_Text

In [361]:
Sansa_Words = []

for word in Sansa_Text:
    if len(word)>=1:
        if word != '0':
            Sansa_Words.append(word)
        
#Sansa_Words 

In [362]:
Sansa_Words_String = ' '.join(Sansa_Words)
#Sansa_Words_String

In [363]:
Sansa_String_Length = len(Sansa_Words_String)
Sansa_String_Length

1706

In [364]:
Sansa_Sentiment = TextBlob(Sansa_Words_String).sentiment
Sansa_Sentiment

Sentiment(polarity=0.07939560439560439, subjectivity=0.49807692307692303)

### Theon Greyjoy

In [365]:
Theon = season_six.loc['Theon']
#print Theon

In [366]:
Theon_Text = Theon.values.tolist()
#Theon_Text

In [367]:
Theon_Words = []

for word in Theon_Text:
    if len(word)>=1:
        if word != '0':
            Theon_Words.append(word)
        
#Theon_Words 

In [368]:
Theon_Words_String = ' '.join(Theon_Words)
#Theon_Words_String

In [369]:
Theon_String_Length = len(Theon_Words_String)
Theon_String_Length

647

In [370]:
Theon_Sentiment = TextBlob(Theon_Words_String).sentiment
Theon_Sentiment

Sentiment(polarity=-0.11041666666666665, subjectivity=0.49083333333333334)

### Tyrion Lannister

In [371]:
Tyrion = season_six.loc['Tyrion']
#print Tyrion

In [372]:
Tyrion_Text = Tyrion.values.tolist()
#Tyrion_Text

In [373]:
Tyrion_Words = []

for word in Tyrion_Text:
    if len(word)>=1:
        if word != '0':
            Tyrion_Words.append(word)
        
#Tyrion_Words 

In [374]:
Tyrion_Words_String = ' '.join(Tyrion_Words)
#Tyrion_Words_String

In [375]:
Tyrion_String_Length = len(Tyrion_Words_String)
Tyrion_String_Length

1979

In [376]:
Tyrion_Sentiment = TextBlob(Tyrion_Words_String).sentiment
Tyrion_Sentiment

Sentiment(polarity=0.15876393110435663, subjectivity=0.54209726443769)

### Varys

In [377]:
Varys = season_six.loc['Varys']
#print Varys

In [378]:
Varys_Text = Varys.values.tolist()
#Varys_Text

In [379]:
Varys_Words = []

for word in Varys_Text:
    if len(word)>=1:
        if word != '0':
            Varys_Words.append(word)
        
#Varys_Words 

In [380]:
Varys_Words_String = ' '.join(Varys_Words)
#Varys_Words_String

In [381]:
Varys_String_Length = len(Varys_Words_String)
Varys_String_Length

800

In [382]:
Varys_Sentiment = TextBlob(Varys_Words_String).sentiment
Varys_Sentiment

Sentiment(polarity=0.20034461152882202, subjectivity=0.5567042606516291)

### Yara Greyjoy

In [383]:
Yara = season_six.loc['Yara']
#print Yara

In [384]:
Yara_Text = Yara.values.tolist()
#Yara_Text

In [385]:
Yara_Words = []

for word in Yara_Text:
    if len(word)>=1:
        if word != '0':
            Yara_Words.append(word)
        
#Yara_Words 

In [386]:
Yara_Words_String = ' '.join(Yara_Words)
#Yara_Words_String

In [387]:
Yara_String_Length = len(Yara_Words_String)
Yara_String_Length

822

In [388]:
Yara_Sentiment = TextBlob(Yara_Words_String).sentiment
Yara_Sentiment

Sentiment(polarity=0.04678571428571429, subjectivity=0.5526190476190476)